In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('punkt_tab')

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression

from sklearn.pipeline import Pipeline

from sklearn.metrics import precision_score, recall_score, precision_recall_curve

from sklearn.preprocessing import LabelEncoder

# Обзор данных

In [2]:
data = pd.read_csv('dataset.txt', delimiter='\t', header=None)
data[['label', 'comment']] = data[0].str.split(' ', n=1, expand=True)
data = data.drop(columns=[0])
data['label'] = data['label'].str.replace('__label__', '', regex=False)

In [3]:
data.head()

,label,comment
0,INSULT,скотина! что сказать
1,NORMAL,я сегодня проезжала по рабочей и между домами ...
2,NORMAL,очередной лохотрон. зачем придумывать очередно...
3,NORMAL,"ретро дежавю ... сложно понять чужое сердце , ..."
4,NORMAL,а когда мы статус агрогородка получили?


Датасет состоит из 2-х столбцов: label - метки комментариев, comment  - комментарии

In [4]:
data.shape

(248290, 2)

In [5]:
data.label.unique()

array(['INSULT', 'NORMAL', 'INSULT,THREAT', 'INSULT,OBSCENITY',
       'OBSCENITY', 'THREAT', 'OBSCENITY,THREAT',
       'INSULT,OBSCENITY,THREAT'], dtype=object)

<br>Уникальные метки: </br>
    <p>INSULT - комментарии, унижающие человека</p>
    <p>NORMAL - нейтральные комментарии пользователей</p>
    <p>OBSCENITY - комментарии, содержащие описание сексуального насилия или угрозу его совершения</p>
    <p>THREAT - комментарии с явным намерением причинить вред другому человеку</p>
    <p>INSULT,THREAT</p>
    <p>INSULT,OBSCENITY</p>
    <p>OBSCENITY,THREAT</p>
    <p>INSULT,OBSCENITY,THREAT</p>

In [6]:
data.label.value_counts()

label
NORMAL                     203685
INSULT                      28567
INSULT,THREAT                6317
THREAT                       5460
OBSCENITY                    2245
INSULT,OBSCENITY             1766
INSULT,OBSCENITY,THREAT       176
OBSCENITY,THREAT               74
Name: count, dtype: int64

Проверка качества разметки:

In [7]:
for c in data[data.label == 'INSULT']['comment'].head():
    print(c)

скотина! что сказать
долбоебы это фэйк
пиздаболы, сделали снимок, придумали историю и подали дурачкам через сми
мляяя..фомин..ты издесь умничаешь,чайка помойная...сука..неужели ты думаешь что уже непопробовали через друзей знвкомых?,
анна склярова (герасименкова) -- защитник я или не не тебе решать, а ты -- точно тупой вонючий тролль и старый пидорас пошёл нах !!!!!!!(tr) 


In [8]:
for c in data[data.label == 'NORMAL']['comment'].head():
    print(c)

я сегодня проезжала по рабочей и между домами снитенко и гомолысовой магазином ( на пустыре) бежала кошка похожего окраса. может, я и ошиблась, но необычный окрас бросился в глаза.
очередной лохотрон. зачем придумывать очередной налог на воздух, если можно обьявить инсульт и грипп- пандемией! и лихо на придурках зарабатывать годами на штрафах, фейковых вакцинах, всевозможных платных тестах, продажей масок и перчаток по баснословным ценам.. самое смешное, что бараны блеют и верят пастуху, телевизору. живут как под гипнозом. не думая, не глядя по сторонам.
ретро дежавю ... сложно понять чужое сердце , лиш ощутить музыкой видимо
а когда мы статус агрогородка получили?
2 августа поздно вечером нашли вот такую потеряшку в районе высоток на победе. девочка явно домашняя, в новом ошейнике. обращаться +7 989 816-43-42


# Предобработка данных

In [9]:
train_data, test_data = train_test_split(data, test_size = 500)

In [10]:
#токенизация 
snowball = SnowballStemmer(language="russian")
russian_stop_words = stopwords.words("russian")

def tokenize_sentence(sentence: str, remove_stop_words: bool = True):
    tokens = word_tokenize(sentence, language="russian")
    tokens = [i for i in tokens if i not in string.punctuation]
    if remove_stop_words:
        tokens = [i for i in tokens if i not in russian_stop_words]
    tokens = [snowball.stem(i) for i in tokens]
    return tokens

In [11]:
vectorizer = TfidfVectorizer(tokenizer=lambda x: tokenize_sentence(x, remove_stop_words=True))


In [12]:
features = vectorizer.fit_transform(train_data["comment"])

C:\INSTALL\anaconda\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


# Модели

Логистическая регрессия:

In [13]:
model = LogisticRegression(random_state=0)
model.fit(features, train_data["label"])

C:\INSTALL\anaconda\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)

In [14]:
model_pipeline = Pipeline([
    ("vectorizer", TfidfVectorizer(tokenizer=lambda x: tokenize_sentence(x, remove_stop_words=True))),
    ("model", LogisticRegression(random_state=0))
]
)

In [15]:
model_pipeline.fit(train_data["comment"], train_data["label"])

C:\INSTALL\anaconda\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\INSTALL\anaconda\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(tokenizer=<function <lambda> at 0x0000015FD105D940>)),
                ('model', LogisticRegression(random_state=0))])

In [16]:
model_pipeline.predict(["Привет, у меня все нормально"])

array(['NORMAL'], dtype=object)

In [17]:
precision_score(y_true=test_data["label"], y_pred=model_pipeline.predict(test_data["comment"]),average='macro')

0.6831167652246083

In [18]:
recall_score(y_true=test_data["label"], y_pred=model_pipeline.predict(test_data["comment"]), average='macro')

0.6045474351356704

xgboost

In [19]:
#Создаем экземпляр LabelEncoder
le = LabelEncoder()

#Преобразуем метки в целые числа
data['label_int'] = le.fit_transform(data['label'])

print(data)

         label                                            comment  label_int
0       INSULT                               скотина! что сказать          0
1       NORMAL  я сегодня проезжала по рабочей и между домами ...          4
2       NORMAL  очередной лохотрон. зачем придумывать очередно...          4
3       NORMAL  ретро дежавю ... сложно понять чужое сердце , ...          4
4       NORMAL            а когда мы статус агрогородка получили?          4
...        ...                                                ...        ...
248285  NORMAL                       правильно всё по пять (5)...          4
248286  INSULT  ёбанные нубы заходите на сервер мой ник _creep...          0
248287  NORMAL  а у меня наверное рекорд в 1962 году в училище...          4
248288  NORMAL                              спасибо всем большое)          4
248289  NORMAL  нельзя ли увеличить хотя бы в два раза некотор...          4

[248290 rows x 3 columns]


In [20]:
label_mapping = dict(zip(le.classes_, range(len(le.classes_))))
print(label_mapping)

{'INSULT': 0, 'INSULT,OBSCENITY': 1, 'INSULT,OBSCENITY,THREAT': 2, 'INSULT,THREAT': 3, 'NORMAL': 4, 'OBSCENITY': 5, 'OBSCENITY,THREAT': 6, 'THREAT': 7}


In [21]:
train_data, test_data = train_test_split(data, test_size = 500)

In [ ]:
import xgboost as xgb
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report

xgbmodel = make_pipeline(TfidfVectorizer(), xgb.XGBClassifier(uselabelencoder=False, eval_metric='mlogloss'))
xgbmodel.fit(train_data["comment"], train_data["label_int"])

ypredxgb = xgbmodel.predict(test_data["comment"])


C:\INSTALL\anaconda\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:38:42] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "uselabelencoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [25]:
precision_score(y_true=test_data["label_int"], y_pred=xgbmodel.predict(test_data["comment"]),average='weighted')

C:\INSTALL\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.8866496112365186

In [26]:
recall_score(y_true=test_data["label_int"], y_pred=xgbmodel.predict(test_data["comment"]), average='weighted')

0.888